In [ ]:
#coding:utf-8
import sys
import numpy as np

sys.path.append("..")
import argparse
from train_models.mtcnn_model import P_Net, R_Net, O_Net
from prepare_data.loader import TestLoader
from Detection.detector import Detector
from Detection.fcn_detector import FcnDetector
from Detection.MtcnnDetector import MtcnnDetector
import cv2
import os

data_dir = '../../DATA/WIDER_val/images'
anno_file = 'wider_face_val.txt'


F:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def read_gt_bbox(raw_list):
    
    list_len = len(raw_list)
    bbox_num = (list_len-1)//4
    idx = 1
    bboxes = np.zeros((bbox_num,4),dtype=int)
    for i in range(4):
        for j in range(bbox_num):
            bboxes[j][i] = int(raw_list[idx])
            idx += 1
    return bboxes
    

In [4]:
def get_image_info(anno_file):
    f = open(anno_file,'r')
    image_info = []
    for line in f:
        ct_list = line.strip().split(' ')
        path = ct_list[0]
        
        path_list = path.split('\\')
        event = path_list[0]
        name = path_list [1]
        #print(event, name )
        bboxes = read_gt_bbox(ct_list)
        image_info.append([event,name,bboxes])
    print('total number of images in validation set: ', len(image_info))
    return image_info

In [5]:
test_mode = "ONet"
thresh = [0.6,0.5,0.4]
min_face_size = 24
stride = 2
slide_window = False
shuffle = False
vis = False
detectors = [None, None, None]
prefix = ['../data/MTCNN_model/PNet_landmark/PNet', '../data/MTCNN_model/RNet_landmark/RNet', '../data/MTCNN_model/ONet_landmark/ONet']
epoch = [18, 14, 16]
batch_size = [2048, 256, 16]
model_path = ['%s-%s' % (x, y) for x, y in zip(prefix, epoch)]


In [6]:

if slide_window:
    PNet = Detector(P_Net, 12, batch_size[0], model_path[0])
else:
    PNet = FcnDetector(P_Net, model_path[0])
detectors[0] = PNet
    
# load rnet model
if test_mode in ["RNet", "ONet"]:
    RNet = Detector(R_Net, 24, batch_size[1], model_path[1])
    detectors[1] = RNet
    
# load onet model
if test_mode == "ONet":
    ONet = Detector(O_Net, 48, batch_size[2], model_path[2])
    detectors[2] = ONet
    
mtcnn_detector = MtcnnDetector(detectors=detectors, min_face_size=min_face_size,
                                   stride=stride, threshold=thresh, slide_window=slide_window)
    

(1, ?, ?, 3)
(1, ?, ?, 10)
(1, ?, ?, 10)
(1, ?, ?, 16)
(1, ?, ?, 32)
(1, ?, ?, 2)
(1, ?, ?, 4)
(1, ?, ?, 10)


../data/MTCNN_model/PNet_landmark/PNet-18


AssertionError: the params dictionary is not valid

In [7]:
image_info = get_image_info(anno_file)

FileNotFoundError: [Errno 2] No such file or directory: 'wider_face_val.txt'

In [96]:
str1='aaa'
str2 = 'bbb'
str3 = 'aaa'
print(str1 != str2)
print (str1 == str3)


True
True


In [ ]:
a ='asdfasdf.jpg'
a.split('.jpg')

['asdfasdf', '']

In [98]:
current_event = ''
save_path = ''
for item in image_info:
    image_file_name = os.path.join(data_dir,item[0],item[1])
    if current_event != item[0]:
        current_event = item[0]
        save_path = os.path.join('../../DATA',item[0])
        if not os.path.exists(save_path):
            os.mkdir(save_path)
    f_name= item[1].split('.jpg')[0]
    dets_file_name = os.path.join(save_path,f_name + '.txt')
    img = cv2.imread(image_file_name)
    all_boxes,_ = mtcnn_detector.detect_single_image(img)
      
        

AttributeError: 'MtcnnDetector' object has no attribute 'detect_single_image'